# IIC-3641 GML UC

- Versiones de librerías, python 3.10.2
- DGL: https://www.dgl.ai/pages/start.html


In [1]:
import torch
print(torch.__version__)

2.4.1+cu118


## DGL requiere de un framework de backend. Aquí va con torch sobre cuda (GPU).

In [2]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import time
import numpy as np

gcn_msg = fn.copy_u(u="h", out="m")
gcn_reduce = fn.sum(msg="m", out="h")

## Leemos el dataset

In [3]:
from dgl.data import CoraGraphDataset


def load_cora_data():
    dataset = CoraGraphDataset()
    g = dataset[0]
    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    test_mask = g.ndata["test_mask"]
    return g, features, labels, train_mask, test_mask

In [4]:
g, features, labels, train_mask, test_mask = load_cora_data()
# Add edges between each node and itself to preserve old node representations
g.add_edges(g.nodes(), g.nodes())

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


## Definimos una capa GCN

In [5]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        with g.local_scope():
            g.ndata["h"] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata["h"]
            return self.linear(h)

## Y definimos la red, en este caso de dos capas

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(1433, 16)
        self.layer2 = GCNLayer(16, 7)

    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x


net = Net()
print(net)


Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [7]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

## Y entrenamos

In [8]:
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []
for epoch in range(100):
    if epoch >= 0:
        t0 = time.time()
    net.train()
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 0:
        dur.append(time.time() - t0)
    acc = evaluate(net, g, features, labels, test_mask)
    print(
        "Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)
        )
    )

Epoch 00000 | Loss 1.9656 | Test Acc 0.1850 | Time(s) 0.0789
Epoch 00001 | Loss 1.8715 | Test Acc 0.3080 | Time(s) 0.0425
Epoch 00002 | Loss 1.7669 | Test Acc 0.4560 | Time(s) 0.0300
Epoch 00003 | Loss 1.6615 | Test Acc 0.4910 | Time(s) 0.0237
Epoch 00004 | Loss 1.5703 | Test Acc 0.5120 | Time(s) 0.0200
Epoch 00005 | Loss 1.4890 | Test Acc 0.5340 | Time(s) 0.0174
Epoch 00006 | Loss 1.4146 | Test Acc 0.5500 | Time(s) 0.0157
Epoch 00007 | Loss 1.3460 | Test Acc 0.5760 | Time(s) 0.0144
Epoch 00008 | Loss 1.2788 | Test Acc 0.6230 | Time(s) 0.0133
Epoch 00009 | Loss 1.2093 | Test Acc 0.6620 | Time(s) 0.0124
Epoch 00010 | Loss 1.1367 | Test Acc 0.6920 | Time(s) 0.0117
Epoch 00011 | Loss 1.0630 | Test Acc 0.7060 | Time(s) 0.0112
Epoch 00012 | Loss 0.9904 | Test Acc 0.6980 | Time(s) 0.0107
Epoch 00013 | Loss 0.9211 | Test Acc 0.6850 | Time(s) 0.0103
Epoch 00014 | Loss 0.8572 | Test Acc 0.6750 | Time(s) 0.0101
Epoch 00015 | Loss 0.7989 | Test Acc 0.6740 | Time(s) 0.0098
Epoch 00016 | Loss 0.744